# Import libraries and helpers

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

In [ ]:
sys.path.append('./helpers_models/')

In [ ]:
sys.path.append('./data_visualization_and_augmentations/')

In [ ]:
sys.path.append('../torch_videovision/')

In [ ]:
sys.path.append('../3D-ResNets-PyTorch/')

In [ ]:
sys.path.append('./important_csvs/')

In [ ]:
from helpers_3d import *

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
from helpers_training import *

# Load Model, change head, freeze body

In [ ]:
options = {
    "model_depth": 50,
    "model": 'resnet',
    "n_classes": 400,
    "n_finetune_classes": 5,
    "resnet_shortcut": 'B',
    "sample_size": (288,352), #(576,704),
    "sample_duration": 16,
    "pretrain_path": '../3D-ResNets-PyTorch/resnet-50-kinetics.pth',
    "no_cuda": False,
    "arch": 'resnet-50',
    "ft_begin_index": 0
}

opts = namedtuple("opts", sorted(options.keys()))

myopts = opts(**options)

model, parameters = generate_model(myopts)

adaptive_pooling = AdaptiveConcatPool3d()
#os.environ['CUDA_VISIBLE_DEVICES']='0,1'
#torch.cuda.empty_cache()
device = torch.device('cuda') 
head = Head()
adaptive_pooling = adaptive_pooling.to(device)
head = head.to(device)
model.module.avgpool = adaptive_pooling
model.module.fc = head

In [ ]:
checkpoint = torch.load('/media/scratch/astamoulakatos/saved-3d-models/best-checkpoint-014epoch.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print('loading pretrained freezed model!')

In [ ]:
for param in model.module.parameters():
    param.requires_grad = False

In [ ]:
# unfreeze 60% of the model
unfreeze(model.module ,0.4)

In [ ]:
check_freeze(model.module)

In [ ]:
tensor_transform = get_tensor_transform('Kinetics', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform(16)
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

In [ ]:
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label = get_indices(df_train, root_dir)
# train_loader = get_loader(16, 18, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader = get_loader(20, 18, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader = get_loader(20, 18, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False, True, 1)


In [ ]:
show_batch(valid_loader, 4)

In [ ]:
inputs, y = next(iter(valid_loader))

In [ ]:
y

In [ ]:
label_smoothing = 0.03

In [ ]:
y_smo = y.float() * (1 - label_smoothing) + 0.5 * label_smoothing

In [ ]:
y_smo

# Confidnce scores of validation set along with labels

In [ ]:
from predictions import *

In [ ]:
model = model.eval()

In [ ]:
device = torch.device('cuda')

In [ ]:
plot_predictions_actuals(valid_loader, 10, model, device, True)

In [ ]:
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        y = y.squeeze(dim=1)
        y = y.float()
        output = model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
from helpers_thresholds import *

In [ ]:
classes = ['Exposure', 'Burial', 'Field Joint', 'Anode', 'Free Span']

In [ ]:
plot_pr_curves(0.0, y_tr, y_pr, classes, '3d_first_unfreezed')

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr, classes)

In [ ]:
thresholds

In [ ]:
df_thresholds = pd.DataFrame({'Threshold': thresholds}).T
df_thresholds

# Table 1

In [ ]:
print(df_thresholds.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

## validation set

In [ ]:
result_valid = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_valid

In [ ]:
print(result_valid.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

## test set

In [ ]:
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        #X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output = model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
result_test = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_test

In [ ]:
print(result_test.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(y_tr, y_pr)

In [ ]:
len(test_loader)

In [ ]:
len(valid_loader)